<a href="https://colab.research.google.com/github/momo2941998/Openpose/blob/master/Openpose_Final_Build_by_Manual_Download_Caffe_Models_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **delete openpose**

In [0]:
rm -rf openpose/

# **set up cmake 3.13.0**

In [0]:
!wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
!tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local

# **clone repo openpose**

In [0]:
!git clone -q --depth 1 https://github.com/CMU-Perceptual-Computing-Lab/openpose.git

# change command suitable

In [0]:
!sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt

# install system dependencies

In [0]:
!apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev

# install python dependencies

In [0]:
!pip install -q youtube-dl

# build openpose
but occur some bugs of caffe's models

In [0]:
!cd openpose && rm -rf build || true && mkdir build && cd build && cmake ..

In [0]:
#!cd openpose/ &&  git submodule update --init --recursive --remote

# Copy models from Drive to fix bugs
NOTE: mount drive before

In [0]:
! cp drive/My\ Drive/openpose_model/pose_iter_584000.caffemodel openpose/models/pose/body_25/
! cp drive/My\ Drive/openpose_model/pose_iter_102000.caffemodel openpose/models/hand/
! cp drive/My\ Drive/openpose_model/pose_iter_116000.caffemodel openpose/models/face/

## and Rebuild with models downloaded

In [0]:
!cd openpose && rm -rf build || true && mkdir build && cd build && cmake ..
!cd openpose/build/ && make -j`nproc`

**import YoutubeVideo**

In [0]:
from IPython.display import YouTubeVideo

# Detect poses on a test video
We are going to detect poses on the following youtube video:

In [0]:
YOUTUBE_ID = 'GZR0mM42dms'


YouTubeVideo(YOUTUBE_ID)

Download the above youtube video, cut the first 30 seconds and do the pose detection on that 30 seconds:

In [0]:
# delete old data
!rm -rf youtube.mp4
!rm -rf openpose/output/
!rm openpose.avi
!mkdir openpose/output/
!mkdir openpose/output/json
!mkdir openpose/output/video

# download the youtube with the given ID
!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID

# cut the first 30 seconds
!ffmpeg -y -loglevel info -i youtube.mp4 -t 30 input.mp4
# detect poses on the these 30 seconds
!cd openpose && ./build/examples/openpose/openpose.bin --video ../input.mp4 --write_json ./output/json/ --display 0  --write_video ../openpose.avi --hand

# convert the result into MP4
!ffmpeg -y -loglevel info -i openpose.avi openpose/output/video/output.mp4

Finally, visualize the result:

In [0]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('openpose/output/video/output.mp4', width=960, height=720)

**And make file zip to download**

In [0]:
!rm -rf output.zip
!zip -r output.zip openpose/output/